# 🚀 RDMA Networking Tutorial for DGX Spark Systems

## What You'll Learn

This notebook will teach you:
1. **What is RDMA?** - The basics explained simply (InfiniBand vs RoCE)
2. **How to check your hardware** - See what's connected
3. **Run speed tests** - Compare RDMA vs TCP/IP
4. **Understand the results** - Why RDMA matters for AI/ML

---

## ⚙️ Part 0: Install Required Packages

Before we can use InfiniBand diagnostic tools, we need to install them. Run the cells below to install all necessary packages.

**What we're installing:**
- `infiniband-diags` - Diagnostic tools (`ibstat`, `ibv_devinfo`, `perfquery`, etc.)
- `rdma-core` - Core RDMA libraries
- `ibverbs-utils` - InfiniBand verbs utilities
- `perftest` - Performance testing tools (`ib_write_bw`, `ib_write_lat`)
- `iperf3` - Network bandwidth testing (works on both Ethernet and InfiniBand)

In [ ]:
# Step 0.1: Install InfiniBand diagnostic and performance tools
# 
# ⚠️ IMPORTANT: Run these commands in a TERMINAL (not in this notebook)
# The notebook cannot prompt for your sudo password.

print("""
╔══════════════════════════════════════════════════════════════════════╗
║  📦 INSTALLATION INSTRUCTIONS                                        ║
║                                                                      ║
║  Copy and paste these commands into a terminal window:               ║
╚══════════════════════════════════════════════════════════════════════╝

# Step 1: Update package list
sudo apt-get update

# Step 2: Install InfiniBand tools
sudo apt-get install -y infiniband-diags rdma-core ibverbs-utils perftest iperf3

══════════════════════════════════════════════════════════════════════════
After running the commands above in a terminal, come back here and run
the NEXT cell (Step 0.2) to verify the installation was successful.
══════════════════════════════════════════════════════════════════════════
""")

In [ ]:
# Step 0.2: Verify installation - check that all tools are available

import shutil

tools_to_check = [
    ("ibstat", "InfiniBand status tool"),
    ("ibv_devinfo", "InfiniBand device info"),
    ("ib_write_bw", "Bandwidth test tool"),
    ("ib_write_lat", "Latency test tool"),
    ("iperf3", "Network bandwidth tester"),
    ("perfquery", "Performance query tool"),
]

print("🔍 Checking installed tools...")
print("=" * 50)

all_found = True
for tool, description in tools_to_check:
    path = shutil.which(tool)
    if path:
        print(f"✅ {tool:15} - {description}")
    else:
        print(f"❌ {tool:15} - NOT FOUND")
        all_found = False

print("=" * 50)
if all_found:
    print("🎉 All tools installed! You're ready to proceed.")
else:
    print("⚠️ Some tools missing. Re-run the installation cell above.")

### ⚠️ Troubleshooting Installation

If installation fails, try running these commands manually in a terminal:

```bash
# Update and install
sudo apt-get update
sudo apt-get install -y infiniband-diags rdma-core ibverbs-utils perftest iperf3

# On some systems, you may also need:
sudo apt-get install -y libibverbs-dev librdmacm-dev
```

**Common issues:**
- "Package not found" → Your system may use different package names. Try `apt search infiniband`
- "Permission denied" → Make sure you have sudo access
- Already installed → Great! Move on to Part 1

---

## 📚 Part 1: Understanding RDMA, InfiniBand, and RoCE

### What is RDMA?

**RDMA (Remote Direct Memory Access)** lets network cards read/write directly to application memory—bypassing the CPU entirely. This is what makes high-speed GPU networking possible.

There are two main ways to get RDMA:

| Technology | InfiniBand | RoCE (RDMA over Converged Ethernet) |
|------------|------------|-------------------------------------|
| Physical Layer | InfiniBand fabric | Standard Ethernet |
| Speed | 100-400 Gbps | 100-400 Gbps |
| Latency | ~1-2 microseconds | ~1-2 microseconds |
| Switches Required | InfiniBand switches | Regular Ethernet switches |
| Configuration | Works out of the box | Needs PFC/ECN on switches |
| **DGX Spark uses** | ❌ | ✅ **Yes** |

### Wait, DGX Spark Uses RoCE?

Yes! If you run `ibv_devinfo`, you'll see device names like `roceP2p1s0f0`. That `roce` prefix tells you this is RoCE, not native InfiniBand.

**The good news**: It doesn't matter for your workloads. Same RDMA benefits, same tools (`ib_write_bw`, etc.), same performance. NCCL doesn't care whether it's InfiniBand or RoCE—it just sees RDMA.

### Why Does This Matter for AI/ML?

When running large AI models across multiple GPUs/machines:
- GPUs need to **constantly share data** (KV-cache, activations, model shards)
- Slow connections = GPUs waiting = wasted expensive hardware
- RDMA keeps GPUs fed with data at maximum speed

### Your Setup: 2 DGX Spark Boxes + 2 RoCE Links

```
┌─────────────────┐           RoCE/RDMA Links         ┌─────────────────┐
│   DGX Spark 1   │ ════════════════════════════════ │   DGX Spark 2   │
│                 │ ════════════════════════════════ │                 │
│  [GPU][GPU]     │      (Super fast connection!)    │  [GPU][GPU]     │
└─────────────────┘                                  └─────────────────┘
```

> 💡 **Note**: The tools are called `ib_*` (e.g., `ib_write_bw`) because they use the InfiniBand verbs API—but they work identically with RoCE.

---

Now let's check what hardware you have! Run the cells below one by one.

## 🔍 Part 2: Check Your System

First, let's import the libraries we need and create some helper functions.

In [ ]:
# Standard imports - run this first!
import subprocess
import os
import sys

def run_command(cmd, description=""):
    """Run a shell command and display the output nicely"""
    if description:
        print(f"\n{'='*60}")
        print(f"📋 {description}")
        print(f"{'='*60}")
        print(f"Command: {cmd}\n")
    
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=30)
        if result.stdout:
            print(result.stdout)
        if result.stderr and result.returncode != 0:
            print(f"⚠️ Error: {result.stderr}")
        return result.returncode == 0
    except subprocess.TimeoutExpired:
        print("⏱️ Command timed out")
        return False
    except Exception as e:
        print(f"❌ Error: {e}")
        return False

print("✅ Helper functions loaded! Ready to explore your system.")

### Step 2.1: What machine am I on?

Let's first check which DGX Spark box you're currently using.

In [ ]:
# Check your hostname and basic system info
run_command("hostname", "Your machine's name (hostname)")
run_command("uname -a", "Operating system details")

### Step 2.2: Do I have InfiniBand hardware?

Let's check if InfiniBand devices are detected on your system.

In [ ]:
# Check for InfiniBand devices
run_command("lspci | grep -i 'infiniband\|mellanox\|connectx'", 
            "Looking for InfiniBand/Mellanox hardware in PCI devices")

**What to expect:** You should see something like "Mellanox ConnectX-7" or similar. This is your RDMA network card (used for both InfiniBand and RoCE)!

### Step 2.3: Check RDMA Device Status

Now let's see the status of your RDMA connections.

In [ ]:
# Check InfiniBand status - this is the main command!
run_command("ibstat", "InfiniBand device status (ibstat)")

### 📖 Understanding ibstat Output

Here's what to look for:

| Field | Good Value | Meaning |
|-------|------------|----------|
| **State** | `Active` | Cable connected and working! |
| **State** | `Down` | No cable or other end not connected |
| **Physical state** | `LinkUp` | Physical connection established |
| **Rate** | `100` or higher | Speed in Gbps |

**If you see `Active` and `LinkUp` → Your InfiniBand cables are connected correctly! 🎉**

In [ ]:
# More detailed view of InfiniBand devices
run_command("ibv_devinfo", "Detailed InfiniBand device information")

### 📖 Understanding ibv_devinfo Output

Look at the device names in the output above. You'll see something like:

- `roceP2p1s0f0` → **This is RoCE** (RDMA over Converged Ethernet)
- `mlx5_0` → Generic Mellanox/NVIDIA device name
- `ib0` → Traditional InfiniBand device name (rare on modern systems)

**If you see `roce` in the device name**: Your system uses RoCE, not native InfiniBand. This is normal for DGX Spark! All the same tools work, and performance is identical.

The key fields to check:
| Field | What to Look For |
|-------|------------------|
| `transport` | `InfiniBand` (even for RoCE, the API is the same) |
| `link_layer` | `Ethernet` (for RoCE) or `InfiniBand` (for native IB) |
| `state` | `PORT_ACTIVE` = ready to use |

### Step 2.4: Check Network Interfaces

InfiniBand creates network interfaces just like Ethernet. Let's see all your network interfaces.

In [ ]:
# Show all network interfaces
run_command("ip link show", "All network interfaces on this system")

In [ ]:
# Show IP addresses assigned to interfaces
run_command("ip addr show", "IP addresses assigned to each interface")

### 📖 Understanding Network Interfaces

You'll likely see several types:

| Interface Name | Type | Description |
|----------------|------|-------------|
| `lo` | Loopback | Internal (127.0.0.1) |
| `eth0`, `enp...` | Ethernet | Regular network cable |
| `ib0`, `ibp...` | **InfiniBand** | Traditional IB interface names (rare now) |
| `enp*np*` (e.g., `enp1s0f0np0`) | **RoCE/RDMA** | Predictable naming for RDMA interfaces |
| `docker0`, `br-...` | Virtual | Docker/container networks |

**On DGX Spark**: Look for interfaces with the `np` suffix (like `enp1s0f0np0`) - these are your RoCE/RDMA interfaces. They look like Ethernet interfaces but support RDMA!

---

## 🎯 Part 3: Understanding Your Two DGX Spark Boxes

Before we run tests, we need to know:
1. The **IP address** of THIS machine (on the RoCE/RDMA interface)
2. The **IP address** of the OTHER machine (on the RoCE/RDMA interface)

Let's find out!

In [ ]:
# Find InfiniBand IP addresses specifically
print("Looking for InfiniBand interfaces (ib0, ib1, enp*np*, etc.)...\n")
run_command("ip addr show | grep -E -A 2 'ib[0-9]|enp.*np[0-9]'", "InfiniBand interface IP addresses")

### 📝 Write Down Your IPs!

Fill in these values based on what you see above:

**This machine (DGX Spark 2):**
- Hostname: `_spark-02___________`
- InfiniBand IP (e.g., enp1s0f0np0): `_192.168.100.11___________`

**Other machine (DGX Spark 1):**
- Hostname: `_spark-01___________`  
- InfiniBand IP (e.g., enp1s0f0np0): `192.168.100.10____________`

*(You'll need to run this notebook on BOTH machines to get both IPs, or SSH to the other machine)*

In [ ]:
# CONFIGURE THIS: Set the IP of the OTHER DGX Spark box
# After running the cells above on both machines, fill in the other machine's InfiniBand IP

OTHER_MACHINE_IB_IP = "192.168.100.10"  # Example: "192.168.1.2" - FILL THIS IN!

if not OTHER_MACHINE_IB_IP:
    print("⚠️ Please set OTHER_MACHINE_IB_IP above!")
    print("   Run the previous cell on your other DGX Spark to find its InfiniBand IP.")
else:
    print(f"✅ Will test connection to: {OTHER_MACHINE_IB_IP}")

---

## 🧪 Part 4: Basic Connectivity Test

Let's make sure the two machines can talk to each other over InfiniBand.

In [ ]:
# Ping test over InfiniBand
if OTHER_MACHINE_IB_IP:
    run_command(f"ping -c 5 {OTHER_MACHINE_IB_IP}", 
                f"Pinging other DGX Spark at {OTHER_MACHINE_IB_IP}")
else:
    print("❌ Please set OTHER_MACHINE_IB_IP first!")

### 📖 Understanding Ping Results

Look at the **time** values:
- `time=0.1 ms` → Excellent! This is InfiniBand speed
- `time=1-5 ms` → Good, still fast
- `time=10+ ms` → Might be going through Ethernet instead

**InfiniBand ping times should be very low (under 1ms typically)**

---

## 📊 Part 5: Speed Tests - The Fun Part!

Now let's measure how fast InfiniBand really is! We'll use tools called `ib_write_bw` and `ib_write_lat`.

### How These Tests Work

```
Machine 1 (Server)          Machine 2 (Client)
┌─────────────────┐         ┌─────────────────┐
│  Listening...   │ ◄────── │  Connect & Test │
│  Ready to       │ ──────► │  Send data back │
│  receive data   │         │  and forth      │
└─────────────────┘         └─────────────────┘
```

**Important:** You need to run commands on BOTH machines!
1. Start the **server** on one machine first
2. Then start the **client** on the other machine

### Step 5.1: Bandwidth Test

**Bandwidth** = How much data can flow per second (like water through a pipe)

#### On Machine 1 (Server) - Run this in a terminal:
```bash
ib_write_bw
```

#### On Machine 2 (Client) - Run the cell below:

In [ ]:
# BANDWIDTH TEST - Run this AFTER starting server on the other machine!
# On the OTHER machine, first run: ib_write_bw

if OTHER_MACHINE_IB_IP:
    print("⏳ Running bandwidth test... (takes about 10 seconds)")
    print("\n🔴 MAKE SURE you ran 'ib_write_bw' on the other machine first!\n")
    run_command(f"ib_write_bw {OTHER_MACHINE_IB_IP}", 
                "InfiniBand Bandwidth Test")
else:
    print("❌ Please set OTHER_MACHINE_IB_IP first!")

### 📖 Understanding Bandwidth Results

The output looks something like this:
```
---------------------------------------------------------------------------------------
 #bytes     #iterations    BW peak[MB/sec]    BW average[MB/sec]   MsgRate[Mpps]
 65536      5000           12045.23           12032.45             0.183521
---------------------------------------------------------------------------------------
```

**Look at the `BW average[MB/sec]` column** - this is your sustained bandwidth.

#### Converting MB/sec to Gbps:
$$\text{Gbps} = \frac{\text{MB/sec} \times 8}{1000}$$

**Why × 8 and ÷ 1000?**

| Step | What it does | Reason |
|------|--------------|--------|
| **× 8** | Converts **Bytes** → **bits** | 1 Byte = 8 bits |
| **÷ 1000** | Converts **Mega** → **Giga** | 1 Giga = 1000 Mega |

```
MB/sec  →  Mb/sec  →  Gb/sec
        ×8         ÷1000
```

- **Benchmark tools** report in **Bytes** (B) - what you actually transfer
- **Network specs** advertise in **bits** (b) - industry standard

This is why a "100 Gbps" InfiniBand link shows ~12,000 MB/sec in benchmarks - same speed, different units!

#### Reference Table:

| BW average (MB/sec) | Calculation | Speed (Gbps) | Hardware |
|---------------------|-------------|--------------|----------|
| ~12,000 MB/sec | 12000 × 8 ÷ 1000 | **~96 Gbps** | ConnectX-6 (100G) |
| ~24,000 MB/sec | 24000 × 8 ÷ 1000 | **~192 Gbps** | ConnectX-6 HDR (200G) |
| ~48,000 MB/sec | 48000 × 8 ÷ 1000 | **~384 Gbps** | ConnectX-7 NDR (400G) |

**Quick mental math:** Divide MB/sec by 125 to get Gbps  
Example: 12,000 ÷ 125 = **96 Gbps** ✅

**For comparison:**
- WiFi 6: ~150 MB/sec (~1.2 Gbps)
- Gigabit Ethernet: ~125 MB/sec (1 Gbps)
- 10G Ethernet: ~1,250 MB/sec (10 Gbps)

**InfiniBand is typically 10-100x faster than standard networking!**

### 📖 ib_write_bw vs ib_send_bw - What's the Difference?

You might see different RDMA bandwidth tests mentioned. Here's what they do:

| Test | RDMA Operation | How it Works |
|------|----------------|--------------|
| `ib_write_bw` | **RDMA Write** | One-sided: sender writes directly to receiver's memory |
| `ib_send_bw` | **RDMA Send** | Two-sided: receiver must post receive buffers first |
| `ib_read_bw` | **RDMA Read** | One-sided: reader pulls data from remote memory |

#### RDMA Write (`ib_write_bw`) - One-Sided

```
Sender                          Receiver
┌──────┐                        ┌──────┐
│ App  │ "Write to addr X"      │ App  │
│      │ ────────────────────►  │      │ (CPU not notified!)
│ RAM  │                        │ RAM  │ ← Data just appears
└──────┘                        └──────┘
```
The sender writes directly to the receiver's memory. The receiver's CPU is never involved—data just appears. **Lowest overhead, best for raw performance testing.**

#### RDMA Send (`ib_send_bw`) - Two-Sided

```
Sender                          Receiver
┌──────┐                        ┌──────┐
│ App  │ "Send this data"       │ App  │ (must pre-post receives)
│      │ ────────────────────►  │      │ ← Gets completion event
│ RAM  │                        │ RAM  │
└──────┘                        └──────┘
```
The receiver must post receive buffers in advance and gets notified when data arrives. **More like traditional message passing (MPI-style).**

#### Which Test to Use?

| Scenario | Recommended Test |
|----------|------------------|
| Validating InfiniBand hardware | `ib_write_bw` (standard) |
| MPI-style workload simulation | `ib_send_bw` |
| Storage/NVMe-oF patterns | `ib_write_bw` or `ib_read_bw` |
| General validation | Both should hit near line-rate |

**Performance difference is typically minimal** - both should achieve similar bandwidth. `ib_write_bw` may show slightly higher numbers due to zero receive-side overhead.

**The full perftest suite:**
- `ib_write_bw` / `ib_write_lat` - RDMA Write operations
- `ib_read_bw` / `ib_read_lat` - RDMA Read operations
- `ib_send_bw` / `ib_send_lat` - RDMA Send operations
- `ib_atomic_bw` / `ib_atomic_lat` - Atomic operations

### Step 5.2: Latency Test

**Latency** = How long it takes for a single message to arrive (like response time)

#### On Machine 1 (Server) - Run this in a terminal:
```bash
ib_write_lat
```

#### On Machine 2 (Client) - Run the cell below:

In [ ]:
# LATENCY TEST - Run this AFTER starting server on the other machine!
# On the OTHER machine, first run: ib_write_lat

if OTHER_MACHINE_IB_IP:
    print("⏳ Running latency test... (takes about 10 seconds)")
    print("\n🔴 MAKE SURE you ran 'ib_write_lat' on the other machine first!\n")
    run_command(f"ib_write_lat {OTHER_MACHINE_IB_IP}", 
                "InfiniBand Latency Test")
else:
    print("❌ Please set OTHER_MACHINE_IB_IP first!")

### 📖 Understanding Latency Results

Look for the **t_avg** (average time) column:

| Latency | What it means |
|---------|---------------|
| 1-2 μs (microseconds) | Excellent! InfiniBand shining |
| 5-10 μs | Good |
| 50+ μs | Something might be wrong |

**For comparison:**
- InfiniBand: ~1-2 microseconds
- Ethernet (local): ~50-200 microseconds  
- WiFi: ~1,000-10,000 microseconds (1-10 ms)

**InfiniBand latency is 50-100x lower than Ethernet!**

---

## 🔗 Part 5.5: Single Cable vs Dual Cable Experiment

Currently, you have **one InfiniBand cable** connecting the two DGX Spark boxes. Let's see what happens when you add a **second cable**!

### Why Use Two Cables?

```
SINGLE CABLE:                          DUAL CABLE:
┌──────────┐    1 cable    ┌──────────┐    ┌──────────┐   2 cables   ┌──────────┐
│  Spark 1 │══════════════│  Spark 2 │    │  Spark 1 │══════════════│  Spark 2 │
│          │              │          │    │          │══════════════│          │
│ Port 1   │              │ Port 1   │    │ Port 1   │              │ Port 1   │
│ Port 2 ○ │  (unused)    │ ○ Port 2 │    │ Port 2   │              │ Port 2   │
└──────────┘              └──────────┘    └──────────┘              └──────────┘
   ~100 Gbps                                  ~200 Gbps (aggregated)
```

**Benefits of dual cables:**
- **Double bandwidth** for large data transfers
- **Redundancy** - if one cable fails, traffic continues
- **Lower latency** for parallel workloads (multiple streams)

### Step 5.5.1: Record Your Single Cable Results

Before adding the second cable, record your current performance:

| Metric | Single Cable Result |
|--------|---------------------|
| `ib_write_bw` (MB/sec) | 88.261 Gbps_____________ |
| `ib_write_lat` (μs) | 1.99 us_____________ |
| `iperf3` (Gbps) | 35 Gbps_____________ |

### Step 5.5.2: Connect the Second Cable

**Physical Setup:**
1. Power down is NOT required (InfiniBand supports hot-plug)
2. Connect the second cable between the unused ports
3. Wait ~10 seconds for the link to come up

**Verify both links are active:**

In [ ]:
# Check that BOTH InfiniBand ports are now active
run_command("ibstat", "Check both InfiniBand ports status")

# You should see TWO ports with State: Active

In [ ]:
# Find IP addresses for BOTH InfiniBand interfaces
print("Looking for both InfiniBand interfaces...\n")
run_command("ip addr show | grep -E -A 2 'enp.*np[0-9]'", "Both InfiniBand interface IPs")

In [ ]:
# Configure the second interface IPs (if not already assigned)
# You'll need to set IPs on BOTH machines for the second interface

# Example IP scheme:
#   Machine 1: enp1s0f0np0 = 192.168.100.10, enp1s0f1np1 = 192.168.101.10
#   Machine 2: enp1s0f0np0 = 192.168.100.11, enp1s0f1np1 = 192.168.101.11

OTHER_MACHINE_IB_IP_2 = "192.168.200.12"  # Fill in the second interface IP of the other machine

print("Set OTHER_MACHINE_IB_IP_2 to the second InfiniBand interface IP of the other machine")
print(f"  First interface IP:  {OTHER_MACHINE_IB_IP}")
print(f"  Second interface IP: {OTHER_MACHINE_IB_IP_2 if OTHER_MACHINE_IB_IP_2 else '(not set)'}")

### Step 5.5.3: Test Bandwidth on Each Link Individually

First, test each cable separately to confirm both are working:

In [ ]:
# Test bandwidth on FIRST cable (same as before)
# On OTHER machine, run: ib_write_bw

if OTHER_MACHINE_IB_IP:
    print("🔗 Testing FIRST InfiniBand link...")
    print("🔴 Run 'ib_write_bw' on the other machine first!\n")
    run_command(f"ib_write_bw {OTHER_MACHINE_IB_IP}", 
                f"Bandwidth Test - Link 1 ({OTHER_MACHINE_IB_IP})")

In [ ]:
# Test bandwidth on SECOND cable
# On OTHER machine, run: ib_write_bw -d rocep1s0f1  (or the second device name)

if OTHER_MACHINE_IB_IP_2:
    print("🔗 Testing SECOND InfiniBand link...")
    print("🔴 Run 'ib_write_bw -d rocep1s0f1' on the other machine first!\n")
    run_command(f"ib_write_bw -d rocep1s0f1 {OTHER_MACHINE_IB_IP_2}", 
                f"Bandwidth Test - Link 2 ({OTHER_MACHINE_IB_IP_2})")
else:
    print("❌ Set OTHER_MACHINE_IB_IP_2 first!")

### Step 5.5.4: Test Aggregated Bandwidth (Both Links Simultaneously)

Now the real test - run traffic on BOTH cables at the same time!

**Method 1: Parallel ib_write_bw tests**

This requires running tests in parallel. Open two terminals on each machine:

```bash
# On Machine 1 (server) - Terminal 1:
ib_write_bw -d rocep1s0f0

# On Machine 1 (server) - Terminal 2:
ib_write_bw -d rocep1s0f1 -p 18516

# On Machine 2 (client) - Terminal 1:
ib_write_bw -d rocep1s0f0 192.168.100.10

# On Machine 2 (client) - Terminal 2:
ib_write_bw -d rocep1s0f1 192.168.200.12 -p 18516
```

**Method 2: iperf3 with parallel streams**

In [ ]:
# Run parallel iperf3 tests on both interfaces
# Start servers on the other machine first:
#   iperf3 -s -p 5201
#   iperf3 -s -p 5202

import subprocess
import threading
import time

def run_iperf_test(ip, port, results, index):
    """Run iperf3 test and store result"""
    try:
        cmd = f"iperf3 -c {ip} -p {port} -t 10 -J"  # JSON output
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=30)
        results[index] = result.stdout
    except Exception as e:
        results[index] = str(e)

if OTHER_MACHINE_IB_IP and OTHER_MACHINE_IB_IP_2:
    print("🔗 Running PARALLEL iperf3 tests on BOTH links...")
    print("🔴 Start two iperf3 servers on the other machine:")
    print("   Terminal 1: iperf3 -s -p 5201")
    print("   Terminal 2: iperf3 -s -p 5202\n")
    
    results = [None, None]
    
    # Create threads for parallel execution
    t1 = threading.Thread(target=run_iperf_test, args=(OTHER_MACHINE_IB_IP, 5201, results, 0))
    t2 = threading.Thread(target=run_iperf_test, args=(OTHER_MACHINE_IB_IP_2, 5202, results, 1))
    
    print("Starting parallel tests...")
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
    print("\n✅ Both tests complete!")
    print("Check the individual results above and add them together for total bandwidth.")
else:
    print("❌ Set both OTHER_MACHINE_IB_IP and OTHER_MACHINE_IB_IP_2 first!")

### 📖 Understanding Dual Cable Results

**Expected Results:**

| Configuration | Bandwidth (ib_write_bw) | Bandwidth (iperf3) |
|--------------|-------------------------|---------------------|
| Single Cable | ~12,000 MB/sec (~96 Gbps) | ~35 Gbps |
| Dual Cable (each) | ~12,000 MB/sec each | ~35 Gbps each |
| Dual Cable (total) | **~24,000 MB/sec (~192 Gbps)** | **~70 Gbps** |

**Key Insights:**

1. **Linear Scaling**: With two independent links, bandwidth approximately doubles
2. **Latency Unchanged**: Adding cables doesn't reduce latency for single messages
3. **Parallelism Required**: To use both links, workloads must send traffic on both interfaces
4. **NCCL Awareness**: NCCL (used by PyTorch/TensorFlow) automatically uses multiple InfiniBand ports when available

### Step 5.5.5: Record Your Dual Cable Results

| Metric | Single Cable | Dual Cable (Total) | Improvement |
|--------|--------------|-------------------|-------------|
| `ib_write_bw` (MB/sec) | 88.252 | 176.5 | 2x |
| `iperf3` (Gbps) | _________ | _________ | _____x |
| `ib_write_lat` (μs) | 1.98 | 4 | 2x |

---

## 🆚 Part 6: Compare InfiniBand vs Ethernet

Let's do a side-by-side comparison using `iperf3` (works on both InfiniBand and Ethernet).

In [ ]:
# Check if iperf3 is installed
run_command("which iperf3", "Checking for iperf3")
run_command("iperf3 --version", "iperf3 version")

### Step 6.1: Test InfiniBand with iperf3

#### On Machine 1 (Server) - Run in terminal:
```bash
iperf3 -s
```

#### ⚠️ Troubleshooting: "Address already in use"

If you get the error `unable to start listener for connections: Address already in use`, the default port 5201 is busy. Try one of these:

```bash
# Option 1: Kill existing iperf3 process
pkill iperf3
iperf3 -s

# Option 2: Use a different port (e.g., 5202)
iperf3 -s -p 5202
# Then on client use: iperf3 -c <IP> -p 5202
```

#### On Machine 2 (Client) - Run the cell below:

In [ ]:
# iperf3 test over InfiniBand
# On the OTHER machine, first run: iperf3 -s

if OTHER_MACHINE_IB_IP:
    print("⏳ Running iperf3 bandwidth test over InfiniBand...")
    print("\n🔴 MAKE SURE you ran 'iperf3 -s' on the other machine first!\n")
    run_command(f"iperf3 -c {OTHER_MACHINE_IB_IP} -p 5202 -t 10", 
                "iperf3 Bandwidth Test (10 seconds)")
else:
    print("❌ Please set OTHER_MACHINE_IB_IP first!")

### 📖 Understanding iperf3 vs ib_write_bw Results

**Why is iperf3 showing ~35 Gbps instead of 100+ Gbps?**

This is **expected behavior**! Here's why:

| Tool | Protocol | What it tests | Expected Speed |
|------|----------|---------------|----------------|
| `ib_write_bw` | **Native RDMA** | True InfiniBand capability | 100-400 Gbps |
| `iperf3` | **TCP/IP over IPoIB** | IP networking over InfiniBand | 20-50 Gbps |

```
ib_write_bw (RDMA):              iperf3 (TCP/IP):
┌─────────────────┐              ┌─────────────────┐
│   Application   │              │   Application   │
│        ↓        │              │        ↓        │
│   Direct RDMA   │ ← Fast!      │   TCP/IP Stack  │ ← Overhead!
│        ↓        │              │        ↓        │
│  InfiniBand HW  │              │  IPoIB Layer    │
└─────────────────┘              │        ↓        │
                                 │  InfiniBand HW  │
                                 └─────────────────┘
```

**Key insight:** 
- **iperf3 at ~35 Gbps** = Your IPoIB (IP-over-InfiniBand) is working well
- **ib_write_bw at 100+ Gbps** = Your native RDMA is working at full speed

**AI/ML frameworks (PyTorch, TensorFlow) use NCCL which leverages native RDMA, not TCP/IP!**

💡 **To improve iperf3 results**, try parallel streams (still won't match RDMA):
```bash
# Server: iperf3 -s
# Client: iperf3 -c <IP> -P 8 -t 10   # 8 parallel streams
```

### Step 6.2: Now Test Regular Ethernet (for comparison)

If your machines also have Ethernet connections, you can compare!

First, find the Ethernet IP of the other machine:

In [ ]:
# Show Ethernet interfaces (not InfiniBand)
run_command("ip addr show | grep -A 2 -E 'eth|enp'", "Ethernet interface IP addresses")

In [ ]:
# OPTIONAL: Set the Ethernet IP of the other machine for comparison
OTHER_MACHINE_ETH_IP = "192.168.100.10"  # Example: "10.0.0.2" - FILL THIS IN!

if OTHER_MACHINE_ETH_IP:
    print("⏳ Running iperf3 bandwidth test over Ethernet...")
    print("\n🔴 MAKE SURE 'iperf3 -s' is still running on the other machine!\n")
    run_command(f"iperf3 -c {OTHER_MACHINE_ETH_IP} -p 5202 -t 10", 
                "iperf3 Bandwidth Test over ETHERNET")
else:
    print("ℹ️ Set OTHER_MACHINE_ETH_IP to compare Ethernet vs InfiniBand")

---

## 🔥 Part 7: NCCL Tests - What Your Inference Stack Actually Uses

All the tests above (`ib_write_bw`, `iperf3`) measure raw network performance. But inference frameworks don't talk to the network directly—they use **NCCL** (NVIDIA Collective Communications Library).

### What is NCCL?

NCCL is NVIDIA's library for GPU-to-GPU communication. When you run tensor parallelism in vLLM, TensorRT-LLM, or any multi-GPU PyTorch workload, NCCL handles the actual data movement.

**Why test NCCL separately?**
- NCCL uses RDMA when available (InfiniBand or RoCE)
- It optimizes communication patterns (all-reduce, all-gather, etc.)
- The performance you see in NCCL tests = performance your inference gets

```
Your Inference Code
       ↓
   PyTorch / vLLM / TensorRT-LLM
       ↓
     NCCL  ← This is what we're testing
       ↓
   RoCE (RDMA over Converged Ethernet)
       ↓
   Other GPUs
```

### Available nccl-tests Benchmarks

| Test | Operation | Use Case |
|------|-----------|----------|
| `all_gather_perf` | Collect data from all GPUs to all GPUs | Tensor parallelism (gathering sharded outputs) |
| `all_reduce_perf` | Sum/reduce across all GPUs | Gradient synchronization, tensor parallelism |
| `broadcast_perf` | One GPU sends to all others | Distributing inputs, model weights |
| `reduce_scatter_perf` | Reduce then scatter results | Data parallelism, ZeRO (Zero Redundancy Optimizer) |
| `sendrecv_perf` | Point-to-point transfers | Pipeline parallelism, KV-cache movement |

> 💡 For this tutorial, we use `all_gather_perf` as the primary benchmark.

### Step 7.1: Check NCCL Installation and Environment

In [ ]:
# Check NCCL version and if it detects InfiniBand
import subprocess

print("🔍 Checking NCCL environment...\n")

# Check if nccl-tests is available
nccl_test_path = subprocess.run("which all_reduce_perf", shell=True, capture_output=True, text=True)
if nccl_test_path.returncode == 0:
    print(f"✅ nccl-tests found: {nccl_test_path.stdout.strip()}")
else:
    print("❌ nccl-tests not found. Install with:")
    print("""
    # Clone and build nccl-tests
    git clone https://github.com/NVIDIA/nccl-tests.git
    cd nccl-tests
    make MPI=1 MPI_HOME=/usr/lib/x86_64-linux-gnu/openmpi
    
    # Or if you have CUDA in a custom location:
    make MPI=1 CUDA_HOME=/usr/local/cuda MPI_HOME=/usr/lib/x86_64-linux-gnu/openmpi
    
    # Add to PATH
    export PATH=$PATH:$(pwd)/build
    """)

# Check PyTorch NCCL
print("\n🔍 Checking PyTorch NCCL backend...")
try:
    import torch
    print(f"✅ PyTorch version: {torch.__version__}")
    print(f"✅ CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"✅ GPU count: {torch.cuda.device_count()}")
        print(f"✅ NCCL available: {torch.distributed.is_nccl_available()}")
except ImportError:
    print("❌ PyTorch not installed")

### Step 8.2: Run NCCL All-Reduce Test (Single Node)

First, let's test NCCL communication between GPUs on the SAME machine. This establishes a baseline.

In [ ]:
# NCCL All-Reduce test on a single node (between local GPUs)
# This uses NVLink/PCIe, not InfiniBand

print("🔄 Running NCCL All-Reduce test (single node)...")
print("This tests GPU-to-GPU communication WITHIN this machine.\n")

# Run all_reduce_perf if available
result = run_command(
    "all_reduce_perf -b 8 -e 128M -f 2 -g 2", 
    "NCCL All-Reduce Performance (2 GPUs, single node)"
)

if not result:
    print("""
If all_reduce_perf is not found, you can test with PyTorch directly.
See the next cell for a PyTorch-based NCCL test.
""")

### Step 8.3: Run NCCL Multi-Node Test (Over RoCE/RDMA)

**This is the real test** - communication between GPUs on DIFFERENT machines over RoCE.

You need to run this with `mpirun` to coordinate processes across both machines.

#### Understanding the mpirun Command

For DGX Spark with 1 GPU per node, use this command format:

```bash
mpirun -np 2 -H 192.168.200.12:1,192.168.200.13:1 \
    --mca plm_rsh_agent "ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no" \
    -x LD_LIBRARY_PATH=$LD_LIBRARY_PATH \
    -x NCCL_DEBUG=INFO \
    $HOME/src/github.com/NVIDIA/nccl-tests/build/all_gather_perf
```

#### mpirun Command Options Reference

| Option | Meaning |
|--------|---------|
| `-np 2` | Total number of MPI (Message Passing Interface) processes (1 per GPU across both nodes) |
| `-H host:n` | Run n processes on each host (use IP addresses if hostnames aren't in /etc/hosts) |
| `--mca` | MCA (Modular Component Architecture) parameter; configures MPI internals |
| `btl_tcp_if_include` | Network interface for MPI byte transfer layer (data transfer) |
| `oob_tcp_if_include` | Network interface for out-of-band MPI control messages (process startup, coordination) |
| `plm_rsh_agent` | PLM (Process Lifecycle Management) remote shell command for launching processes; default is `ssh`, can add options to skip host key prompts |
| `-x VAR` | Export environment variable to remote processes |

#### nccl-tests Options

| Option | Meaning |
|--------|---------|
| `-b 8` | Start message size (8 bytes) |
| `-e 128M` | End message size (128 MB) |
| `-f 2` | Step factor: multiply size by 2 each iteration |
| `-g 1` | GPUs per process |
| `-n 20` | Number of iterations (default: 20) |
| `-w 5` | Warmup iterations (default: 5) |
| `-c 1` | Check correctness of results |

#### What to Look For in the Output

**1. Transport Selection (in NCCL_DEBUG output):**
```
NCCL INFO NET/IB : Using [0]rocep1s0f0:1/RoCE [1]rocep1s0f1:1/RoCE [2]roceP2p1s0f0:1/RoCE [3]roceP2p1s0f1:1/RoCE
NCCL INFO Using network IB
```
`NET/IB` confirms NCCL found and is using RDMA over the RoCE interfaces.

If you see `NET/Socket` instead, it's falling back to TCP (requires troubleshooting).

**2. Channels established over RDMA:**
```
NCCL INFO Channel 00/0 : 0[0] -> 1[0] [send] via NET/IB/4
NCCL INFO Channel 01/0 : 0[0] -> 1[0] [send] via NET/IB/5
```

**3. Bandwidth Results:**
```
#       size      time   algbw   busbw
#        (B)      (us)  (GB/s)  (GB/s)
    33554432   1020.46   32.88   16.44
# Avg bus bandwidth    : 15.9891
```
- **algbw** = Algorithm bandwidth (raw throughput: message_size / time)
- **busbw** = Bus bandwidth (normalized for the algorithm's data movement pattern)

For all-gather on 2 GPUs over dual 100G RoCE links, expect **busbw of 15-22 GB/s** for large messages.

In [ ]:
# Generate the mpirun command for multi-node NCCL test
import os

# Configuration - update these for your setup
SPARK1_IP = "192.168.200.12"  # spark-01 RoCE IP
SPARK2_IP = "192.168.200.13"  # spark-02 RoCE IP
NCCL_TESTS_PATH = os.path.expandvars("$HOME/src/github.com/NVIDIA/nccl-tests/build")

print("=" * 70)
print("📋 NCCL MULTI-NODE TEST INSTRUCTIONS")
print("=" * 70)

print(f"""
This test requires running mpirun from a terminal (not this notebook)
because it needs to coordinate processes across both machines.

PREREQUISITE: SSH access between machines (password or key-based)
  ssh-keygen -t rsa  # if you don't have keys
  ssh-copy-id {SPARK2_IP}

══════════════════════════════════════════════════════════════════════
TEST 1: All-Gather (default message sizes)
══════════════════════════════════════════════════════════════════════

Run from spark-01:
""")

print(f"""mpirun -np 2 -H {SPARK1_IP}:1,{SPARK2_IP}:1 \\
    --mca plm_rsh_agent "ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no" \\
    -x LD_LIBRARY_PATH=$LD_LIBRARY_PATH \\
    -x NCCL_DEBUG=INFO \\
    {NCCL_TESTS_PATH}/all_gather_perf
""")

print("""
══════════════════════════════════════════════════════════════════════
TEST 2: All-Gather with 16 GB message (maximum throughput test)
══════════════════════════════════════════════════════════════════════
""")

print(f"""mpirun -np 2 -H {SPARK1_IP}:1,{SPARK2_IP}:1 \\
    --mca plm_rsh_agent "ssh -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no" \\
    -x LD_LIBRARY_PATH=$LD_LIBRARY_PATH \\
    -x NCCL_DEBUG=INFO \\
    {NCCL_TESTS_PATH}/all_gather_perf -b 16G -e 16G -f 2
""")

print("""
══════════════════════════════════════════════════════════════════════
WHAT TO LOOK FOR IN THE OUTPUT
══════════════════════════════════════════════════════════════════════

✅ GOOD - Using RDMA (RoCE):
   "NCCL INFO NET/IB : Using [0]rocep1s0f0:1/RoCE [1]rocep1s0f1:1/RoCE..."
   "NCCL INFO Using network IB"

❌ BAD - Falling back to TCP:
   "NCCL INFO NET/Socket"

EXPECTED BANDWIDTH (busbw column):
  - 32 MB message:  ~16 GB/s
  - 16 GB message:  ~22 GB/s (dual 100G links saturated)
  - < 5 GB/s for large messages = RDMA not working, troubleshoot!
""")

print("=" * 70)

### 📖 Understanding NCCL Test Results

#### Key Indicators in NCCL_DEBUG Output

When running with `NCCL_DEBUG=INFO`, look for these four key indicators:

**1. Both GPUs detected across nodes:**
```
#  Rank  0 Group  0 Pid 144333 on   spark-01 device  0 [000f:01:00] NVIDIA GB10
#  Rank  1 Group  0 Pid  69466 on   spark-02 device  0 [000f:01:00] NVIDIA GB10
```
This confirms NCCL sees both GPUs. The `[000f:01:00]` is the PCIe bus ID.

**2. RDMA transport selected (not TCP fallback):**
```
NCCL INFO NET/IB : Using [0]rocep1s0f0:1/RoCE [1]rocep1s0f1:1/RoCE [2]roceP2p1s0f0:1/RoCE [3]roceP2p1s0f1:1/RoCE
NCCL INFO Using network IB
```
`NET/IB` confirms NCCL is using RDMA over the RoCE interfaces. All four interfaces are detected (dual-port on each node).

⚠️ **If you see `NET/Socket` instead**, NCCL is falling back to TCP. This requires troubleshooting:
- Check that RoCE interfaces have IP addresses configured
- Verify `ibstat` shows ports as Active
- Ensure no firewall is blocking RDMA traffic

**3. Channels established over RDMA:**
```
NCCL INFO Channel 00/0 : 0[0] -> 1[0] [send] via NET/IB/4
NCCL INFO Channel 01/0 : 0[0] -> 1[0] [send] via NET/IB/5
```
NCCL creates multiple channels for parallelism. `NET/IB/4` and `NET/IB/5` refer to different RoCE interfaces, showing NCCL is load-balancing across both 100G links.

**4. Performance results show expected bandwidth:**
```
#       size      time   algbw   busbw
#        (B)      (us)  (GB/s)  (GB/s)
    33554432   1020.46   32.88   16.44
# Avg bus bandwidth    : 15.9891
```

---

#### Actual Results from DGX Spark

**32 MB message:**
```
#       size      time   algbw   busbw
#        (B)      (us)  (GB/s)  (GB/s)
    33554432   1020.46   32.88   16.44
# Avg bus bandwidth    : 15.9891
```

**16 GB message (maximum throughput):**
```
#       size      time   algbw   busbw
#        (B)      (us)  (GB/s)  (GB/s)
 17179869184   387603   44.32   22.16
# Avg bus bandwidth    : 21.9736
```

---

#### Understanding the Columns

| Column | What it means |
|--------|---------------|
| `size` | Message size in bytes |
| `time` | Time to complete operation (microseconds) |
| `algbw` | Algorithm bandwidth = size / time (raw throughput) |
| `busbw` | Bus bandwidth = algbw × (N-1)/N for all-gather (actual link utilization) |

**Why busbw differs from algbw?**

For all-gather with N GPUs, each GPU sends its data to all other GPUs. The bus bandwidth formula `busbw = algbw × (N-1)/N` normalizes for this pattern, giving a better measure of actual network utilization.

With 2 GPUs: busbw = algbw × 0.5. So if algbw is 44 GB/s, busbw is ~22 GB/s.

---

#### Expected Performance Benchmarks

| Message Size | Expected busbw | Notes |
|--------------|----------------|-------|
| < 1 KB | Low | Latency-bound, not meaningful for bandwidth |
| 32 MB | ~16 GB/s | Single 100G link saturated |
| 16 GB | ~22 GB/s | Both 100G links working together |
| < 5 GB/s (large msg) | ⚠️ Problem | NCCL likely not using RDMA |

### Step 7.4: Alternative - PyTorch NCCL Test

If nccl-tests isn't installed, you can verify NCCL over InfiniBand with PyTorch directly.

Save this as `nccl_test.py` and run with `torchrun`:

In [ ]:
# Create a simple PyTorch NCCL benchmark script
nccl_test_script = '''#!/usr/bin/env python3
"""
Simple NCCL All-Reduce benchmark using PyTorch.
Tests GPU-to-GPU communication bandwidth.

Run with:
  # Single node (2 GPUs):
  torchrun --nproc_per_node=2 nccl_test.py
  
  # Multi-node (requires MASTER_ADDR, MASTER_PORT):
  # On node 1:
  torchrun --nproc_per_node=2 --nnodes=2 --node_rank=0 \\
           --master_addr=<node1_ip> --master_port=29500 nccl_test.py
  # On node 2:
  torchrun --nproc_per_node=2 --nnodes=2 --node_rank=1 \\
           --master_addr=<node1_ip> --master_port=29500 nccl_test.py
"""
import os
import time
import torch
import torch.distributed as dist

def benchmark_all_reduce(size_mb, num_iterations=100):
    """Benchmark all-reduce for a given tensor size."""
    rank = dist.get_rank()
    world_size = dist.get_world_size()
    device = torch.device(f"cuda:{rank % torch.cuda.device_count()}")
    
    # Create tensor
    num_elements = (size_mb * 1024 * 1024) // 4  # float32 = 4 bytes
    tensor = torch.randn(num_elements, device=device)
    
    # Warmup
    for _ in range(10):
        dist.all_reduce(tensor)
    torch.cuda.synchronize()
    
    # Benchmark
    start = time.perf_counter()
    for _ in range(num_iterations):
        dist.all_reduce(tensor)
    torch.cuda.synchronize()
    elapsed = time.perf_counter() - start
    
    # Calculate bandwidth
    # All-reduce moves 2*(n-1)/n * size data
    bytes_moved = size_mb * 1024 * 1024 * 2 * (world_size - 1) / world_size
    total_bytes = bytes_moved * num_iterations
    bandwidth_gbps = (total_bytes / elapsed) / 1e9
    
    return bandwidth_gbps, elapsed / num_iterations * 1000  # GB/s, ms

def main():
    dist.init_process_group(backend="nccl")
    rank = dist.get_rank()
    world_size = dist.get_world_size()
    
    if rank == 0:
        print(f"\\n{'='*60}")
        print(f"NCCL All-Reduce Benchmark")
        print(f"World size: {world_size} GPUs")
        print(f"{'='*60}")
        print(f"{'Size':>10} {'Bandwidth':>15} {'Latency':>15}")
        print(f"{'(MB)':>10} {'(GB/s)':>15} {'(ms)':>15}")
        print("-" * 60)
    
    for size_mb in [1, 8, 32, 128, 512]:
        bw, latency = benchmark_all_reduce(size_mb)
        if rank == 0:
            print(f"{size_mb:>10} {bw:>15.2f} {latency:>15.3f}")
    
    if rank == 0:
        print("=" * 60)
        print("\\nExpected bandwidth over InfiniBand: 20-40+ GB/s for large messages")
        print("If seeing < 5 GB/s, check NCCL_DEBUG=INFO output for transport type")
    
    dist.destroy_process_group()

if __name__ == "__main__":
    main()
'''

# Save the script
script_path = "/home/nvidia/src/github.com/elizabetht/spark/infiniband-tutorial/nccl_test.py"
with open(script_path, "w") as f:
    f.write(nccl_test_script)

print(f"✅ Saved NCCL test script to: {script_path}")
print("""
To run single-node test (2 GPUs on this machine):
  torchrun --nproc_per_node=2 nccl_test.py

To run multi-node test over InfiniBand:
  # On machine 1 (master):
  NCCL_DEBUG=INFO torchrun --nproc_per_node=2 --nnodes=2 --node_rank=0 \\
      --master_addr={0} --master_port=29500 nccl_test.py
  
  # On machine 2:
  NCCL_DEBUG=INFO torchrun --nproc_per_node=2 --nnodes=2 --node_rank=1 \\
      --master_addr={0} --master_port=29500 nccl_test.py
""".format(OTHER_MACHINE_IB_IP if 'OTHER_MACHINE_IB_IP' in dir() else '<master_ip>'))

### 🔧 Troubleshooting NCCL Over InfiniBand

**NCCL isn't using InfiniBand?** Check these environment variables:

```bash
# Force NCCL to use InfiniBand
export NCCL_IB_DISABLE=0

# Specify which InfiniBand devices to use  
export NCCL_IB_HCA=mlx5_0,mlx5_1

# Enable debug output to see transport selection
export NCCL_DEBUG=INFO
export NCCL_DEBUG_SUBSYS=INIT,NET
```

**Common Issues:**

| Symptom | Likely Cause | Fix |
|---------|--------------|-----|
| "NET/Socket" in debug | IB not detected | Check `ibstat`, ensure ports are Active |
| Low bandwidth (< 5 GB/s) | Using TCP fallback | Set `NCCL_IB_DISABLE=0` |
| "No route to host" | Network config | Ensure IB IPs are reachable (`ping`) |
| "Bootstrap failed" | Firewall | Open ports 29500+ or disable firewall |

### Step 7.5: Record Your NCCL Results

In [ ]:
# Record your NCCL test results
nccl_results = {
    "test_type": "all_reduce_perf / PyTorch",  # Which test did you run?
    "num_gpus": 4,  # Total GPUs across all nodes
    "num_nodes": 2,  # Number of machines
    "transport_used": "IB",  # Check NCCL_DEBUG output: "IB" or "Socket"?
    "bandwidth_128mb_gbps": 0,  # busbw for 128MB message
    "notes": ""
}

print("📊 NCCL Test Results:")
print("-" * 40)
for key, value in nccl_results.items():
    print(f"  {key}: {value}")
print("-" * 40)
print("""
Fill in your results above after running the NCCL tests!

Key question: Did NCCL_DEBUG show "NET/IB" or "NET/Socket"?
- NET/IB = InfiniBand is working ✅
- NET/Socket = Falling back to TCP ❌
""")

---

## 📝 Record Your Results Here

Fill in your actual test results:

In [ ]:
# Record your results here for reference
my_results = {
    "machine_1_hostname": "spark-01",  # Fill in
    "machine_2_hostname": "spark-02",  # Fill in
    "infiniband_bandwidth_gbps":  96.3,  # From ib_write_bw test
    "infiniband_latency_us": 1.96,  # From ib_write_lat test
    "ethernet_bandwidth_gbps": 42.25,  # From iperf3 over ethernet (if tested)
    "notes": ""
}

print("📊 Your recorded results:")
for key, value in my_results.items():
    print(f"  {key}: {value}")